# Assignment 1 Recommender Systems Matrix Factorization

Group 27

Our task was to create recommender systems using the 1M dataset from Movielens. We implemented four naive approaches and matrix facotrization.

We implemented, in Python, several recommendation algorithms and estimate their accuracy with the Root Mean Squared Error (RMSE), and the Mean  Absolute  Error (MAE).  In addition, to  make  sure  that  your  results  are  reliable  use  5-fold  cross-validation.  The average error of these five models (measured on the 5 test sets) 
is a reliable estimate of the accuracy of the (hypothetical) final model that is trained on the whole data set. 5-fold cross-validation is described in  more detail later.

In [1]:
import numpy as np
from sklearn import linear_model
import pandas as pd
import time

## The Data

The data contains 1,000,209 anonymous ratings of approximately 3,900 movies made by 6,040 MovieLens users who joined MovieLens in 2000. format is UserID::MovieID::Rating::Timestamp


In [2]:
#ratings = pd.read_csv('ratings.dat')

ratings = np.loadtxt('ratings.dat', delimiter="::")

#format is UserID::MovieID::Rating::Timestamp
users = np.unique(ratings[:,0])
items = np.unique(ratings[:,1])	

## Matrix Factorization

The goal of matrix factorization is to find two matrices U and M that when you multiply them together you get your original matrix X. We apply matrix facotization to predict ratings in collaborative filtering. First represent all the existing ratings as a matrix. It will be a sparse matrix becasue not every user rates every movie. We must fill in the missing rating for each user. In ways that are consistent with existing ratings. 

We will call our set of users, U and set of items M. Let X of size |U|×|M| be the matrix that contains all the ratings that the users have assigned to the items. We need to first find two matrics matrices P (of size |U|×|K|) and Q (of size |M|×|K|) where K is some latent features such that their product apprioximates |X|.

In this way, each row of P
would represent the strength of the associations between a user and the features. Similarly, each row of Q would represent the strength of the associations between an item and the features. To get the prediction of a rating of an item $m_{j}$ by $u_{i}$, we can calculate the dot product of their vectors:
\begin{equation*}
\hat{x}_{ij} = p^{T}_{i}* q_{j}=∑^{k}_{k=1}p_{ik}q_{kj}
\end{equation*}
Now, we have to find a way to obtain P and Q. We use the Gradient Descent which first finds difference between the matrix P*Q then tries ot minimize the difference iteratively. The difference here, called the error between the estimated rating and the real rating, can be calculated by the following equation for squared error because the estimated rating can be either higher or lower than the real rating:

\begin{equation*}
e^{2}_{ij} = (r_{ij}−\hat{r}_{ij})^{2} = (r_{ij} − ∑^{K}_{k=1}p_{ik}q{kj})^{2}
\end{equation*}
 
Then to minimize the error, we have to find the direction we have to modify the values of $p_{ik}$
and $q_{kj}$. So we need to know the gradient at the current values, and to find that we differentiate the above equation with respect to these two variables separately:

\begin{equation*}
\frac{\partial}{\partial p_{ik}}e^{2}_{ij} = −2(r_{ij}−\hat{r}_{ij})(q_{kj})=−2e_{ij}q_{kj}
\end{equation*}

\begin{equation*}
\frac{\partial}{\partial q_{ik}}e^{2}_{ij} = −2(r_{ij}−\hat{r}_{ij})(p_{kj})=−2e_{ij}p_{kj}
\end{equation*}

Having obtained the gradient, we can now formulate the update rules for both $p_{ik}$
and $q_{kj}$:

\begin{equation*}
p′_{ik} = p_{ik}+ \alpha \frac{\partial}{\partial p_{ik}}e^{2}_{ij} = p_{ik}+2 \alpha e_{ij}q_{kj}
\end{equation*}

\begin{equation*}
q′_{kj} = q_{kj}+\alpha \frac{\partial}{\partial q_{ik}}e^{2}_{ij} = q_{kj}+2 \alpha e_{ij}p_{ik}
\end{equation*}

Where $\alpha$ is a constant whose value determines the rate of approaching the minimum. Usually we will choose a small value for $\alpha$, say 0.005. This is because if we make too large a step towards the minimum we may run into the risk of missing the minimum and end up oscillating around the minimum. We came up with the values for the learn rate and bias through trial and error and reseraching the results given on the MyMediaLite.net 

In [3]:
start_time = time.time()
print('Matrix Factorization Parameters: number_factors=10, number_iterations=75, reg=0.05, learn_rate=0.005, np.random.seed(17)')

Matrix Factorization Parameters: number_factors=10, number_iterations=75, reg=0.05, learn_rate=0.005, np.random.seed(17)


In [4]:
def gravAlgorithm(train, test, num_factors=10, reg=0.35, n_iter=75, learn_rate=0.005):
    """
    This function implements the gravity algorithm described in the paper
    We uaed online research to determine the parameters
    """
    
    train=np.array(train)
    test=np.array(test)
    
    # Initializing from the standard normal dist
    U = np.random.rand(max(np.max(train[:,0]), np.max(test[:,0]) + 1), num_factors)
    M = np.random.rand(num_factors,max(np.max(train[:,1]),np.max(test[:,1])) + 1)
    
    for i in range(n_iter):
        for j in range(len(train)):
            e_grad = 2 * (train[j,2] - np.dot(U[train[j,0],:], M[:,train[j,1]]))
            
            # calculate the gradient of e**2 to M before changing U or the negative of gradient
            
            m_grad = e_grad * U[train[j,0],:]
            u_grad = e_grad * M[:,train[j,1]]
            U[train[j, 0], :] += learn_rate * (u_grad - reg * U[train[j, 0], :])
            M[:, train[j, 1]] += learn_rate * (m_grad - reg * M[:,train[j, 1]])

        # calculate estimated ratings
        est_R = np.dot(U,M)

        # prediction for train
        predictionTrain = np.zeros(len(train))
        for i in range(len(train)):
            
            predictionTrain[i] = est_R[train[i, 0], train[i, 1]]
            
            if predictionTrain[i] > 5:
                
                predictionTrain[i] = 5
                
            if predictionTrain[i] < 1:
                
                predictionTrain[i] = 1

        # prediction for test
        prediction = np.zeros(len(test))
        for i in range(len(test)):
            
            prediction[i] = est_R[test[i,0], test[i, 1]]
            
            if prediction[i] > 5:
                
                prediction[i] = 5
                
            if prediction[i] < 1:
                
                prediction[i] = 1
        
        return (predictionTrain, prediction)

We used a pandas dataframe to store the ratings and in a way that would make the matrix calculations more efficent. 
Then we allocated memory to store the errors for the test and the training data. We also ustilized the five-fold cross-validation methods and calculated the error for each fold and then averaged over it.

In [5]:
def matrix_fact(ratings):

    """
    This function performs the matrix factorization and calcualtes the error. 
    Inputs: The ratings array, and number of interations for testing
    """
    ratings_user=pd.DataFrame(ratings)

    nfolds = 5
    
    #Define the memory for the code
    error_train = np.zeros(nfolds)
    error_test = np.zeros(nfolds)
    train_MAE = np.zeros(nfolds)
    test_MAE = np.zeros(nfolds)

    np.random.seed(17)

    seqs=[x%nfolds for x in range(len(ratings))]
    np.random.shuffle(seqs)

    for fold in range(nfolds):
        
        train_set = np.array([x!=fold for x in seqs])
        test_set = np.array([x==fold for x in seqs])
        train = pd.DataFrame(ratings_user.iloc[train_set], columns=[0, 1, 2], dtype=int)
        test = pd.DataFrame(ratings_user.iloc[test_set], columns=[0, 1, 2], dtype=int)
        
        #pred = gravAlgorithm(train, test, num_factors=10, num_iter=75, reg=0.05, learn_rate=0.005)
        #Calculate the errors RMSE and MAE
        error_train[fold] = np.sqrt(np.mean((np.array(train[2]) - gravAlgorithm(train, test, num_factors=10, reg=0.05, \
                                                                                n_iter=75, learn_rate=0.005)[0])**2))
        
        train_MAE[fold] = np.mean(np.abs(np.array(train[2]) - gravAlgorithm(train, test, num_factors=10, reg=0.05, \
                                                                            n_iter=75, learn_rate=0.005)[0]))
    
        error_test[fold]= np.sqrt(np.mean((np.array(test[2]) - gravAlgorithm(train, test, num_factors=10, reg=0.05, \
                                                                             n_iter=75, learn_rate=0.005)[1])**2))
        
        test_MAE[fold] = np.mean(np.abs(np.array(test[2]) - gravAlgorithm(train, test, num_factors=10, reg=0.05, \
                                                                          n_iter=75, learn_rate=0.005)[1]))
    
        #Print error on each for sanity check
        print("Fold " + str(fold+1) + ": train_RMSE = " + str(error_train[fold]) + " and test_RMSE = " + str(error_test[fold]))
    
    #Print the results
    print("\n")
    print("Mean error on train set: " + str(np.mean(error_train)))
    print("Mean error on  test set: " + str(np.mean(error_test)))
    print('MAE on training set: ' + str(np.mean(train_MAE)))
    print('MAE on  test set: ' + str(np.mean(test_MAE)))
    print("Matrix factorization runtime:  %s seconds ---" % (time.time() - start_time))
    

In [6]:
matrix_fact(ratings)

Fold 1: train_RMSE = 0.978042798891 and test_RMSE = 0.990214312806
Fold 2: train_RMSE = 0.972439425233 and test_RMSE = 0.99282586479
Fold 3: train_RMSE = 0.975312214736 and test_RMSE = 0.988785235883
Fold 4: train_RMSE = 0.978499137938 and test_RMSE = 0.992343315675
Fold 5: train_RMSE = 0.978593153238 and test_RMSE = 0.989851496792


Mean error on train set: 0.976577346007
Mean error on  test set: 0.990804045189
MAE on training set: 0.755578998821
MAE on  test set: 0.767784276863
Matrix factorization runtime:  308.01485800743103 seconds ---


# Results

The Results we as follows:

Fold 1: RMSE_train = 0.978042798891, RMSE_test = 0.990214312806
Fold 2: RMSE_train = 0.972439425233, RMSE_test = 0.99282586479
Fold 3: RMSE_train = 0.975312214736, RMSE_test = 0.988785235883
Fold 4: RMSE_train = 0.978499137938, RMSE_test = 0.992343315675
Fold 5: RMSE_train = 0.978593153238, RMSE_test = 0.989851496792


Mean error on TRAIN: 0.976577346007
Mean error on  TEST: 0.990804045189
MAE on TRAIN: 0.755578998821
MAE on  TEST: 0.755578998821
Matrix factorization runtime:  307.3645968437195 seconds ---


The results produced by our matrix factorization showed to match the results provided by online literature to a certain extent (https://www.cs.uic.edu/~liub/KDD-cup-2007/proceedings/gravity-Tikk.pdf).
The RMSE achieved is not quite as low as comparable results, but in total produce similar results.
The matrix factorization proved to be more accurate than most naive approaches. However, the linear regression approach achieved a lower RMSE which is an unexpected find. I believe our matrix factorization approach could be improved upon to make it more accurate.

Concluding it can be said that matrix factorization is an useful technique for problems as the one at hand, but at the same time is  computationally expensive, especially if a larger number of folds is required to achieve satisfying results. 

## Computational Costs

This algorithm took a much longer time then the linear regression. We estimated the time complexity to be O(R^{2}) to account for going through all the ratings. The memory required for this algorithm is the memory needs to create the the estimation matrix so O(est_R) which in is O(UxM).